# TIMESERIES TO IMAGE CONVERTER

In [1]:
from nilmtk import DataSet
import numpy as np

from utils.log import print_parameters, print_progress, print_log, print_end, print_end_of_loop
from utils.init import get_appliances, param_setup
from utils.file_handling import store_many_hdf5, create_file
from utils.data_handling import mount_data, append_images
from utils.filters import filter_empty_slices_and_fill_missing_samples, filter_low_entropy_slices
from utils.process import get_state, moving_window

dataset_name = "iawe"
dataset = DataSet('datasets/'+dataset_name+'.h5')


Bad key "text.kerning_factor" on line 4 in
/Users/jakob/anaconda3/envs/nilmtk-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/jakob/anaconda3/envs/nilmtk-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
par = {
    'step_in_mins': 13, # Window size
    'max_images': 10000000, # Number of images per appliance per building.
    'img_size': 100, # Output image size.
    'frames': 1, # Video frames
    'allowed_delta_between_frames': 600, # Allowed time difference between frames.
    'resample_period': 6, # Resamples time series data to given resample period. Sample period of dataset can be found in dataset.metadata
    'fill_limit': 10, # Limit of how many samples to backfill when resampling, larger back fill should yield larger output. 
    'percentage_of_missing_data_allowed': 0.85, # Missing data will be backfilled.
    'add_brightness': True, # Multiply non-zero mean power to image.
    'sig_save': True, # Save source power signal.
    'ts_save': True, # Save source time stamps.
    'state_save': True, # Save state of slice (on or off).
    'multiple_buildings': True, # If false it processes only one building.
    'selected_building': 1, # Used if parameter above is False.
    'manually_select_appliances': False # Can be set in get_appliances() function.
    }

In [3]:
# Fixes possible misconfiguration and obtains metadata.
param_setup(dataset,par)

par["appliances"] = get_appliances(dataset,par)
print_parameters(par)
file_name = create_file(par)

# Define global metrics.
healthy_appliances = set()
sig_stacked = 0 
sig_stacked_per_appliance = 0 

# Collect at least "max_dataset_size" images for each appliance for every building.
for appliance in par["appliances"]:
    print_log(par,"\n","Starting "f"{appliance} ("f"{par['appliances'].index(appliance)+1}/"f"{len(par['appliances'])}):")

    # Define metric.
    sig_stacked_per_appliance = 0
    
    # Loop through all buildings.
    for building in dataset.buildings:
        print_log(par,"\n","Starting building "f"{building}")

        # Define temporary array to store image / frames. 
        sig_stack_tmp = np.zeros([0, par["ts_size"]])
        timestamp_stack_tmp = np.zeros([0, par["ts_size"]])
        state_stack_tmp = np.zeros(0)
        # Define main array for video to store.
        sig_stack = np.zeros([0, par["frames"], par["ts_size"]])
        timestamp_stack = np.zeros([0, par["frames"], par["ts_size"]])
        state_stack = np.zeros([0,par["frames"]])

        # Use only selected building.
        if par["multiple_buildings"] == False:
            if int(building)  != par["selected_building"]:
                print_log(par,"skipping building "f"{building} due to parameter multiple_buildings ")
                continue
        
        # Filter out labels (appliances) with appliance.
        for meter in dataset.buildings[building].elec.submeters().meters:  
            
            # Get appliance name.
            label = meter.appliances[0].metadata.get("type")
            
            # Continue only for appliance from the main loop.
            if label != appliance : continue 

            # Load data into RAM.
            signal,time_stamps = mount_data(meter, par)

            # Slice time stamps and signal data to specified length.
            time_stamps_slices = moving_window(time_stamps, par["ts_size"])
            signal_slices = moving_window(signal, par["ts_size"])

            print("1",signal_slices.shape)
            # Filter out low entropy data. 
            signal_slices, time_stamps_slices = filter_empty_slices_and_fill_missing_samples(signal_slices, time_stamps_slices, par)

            print("2",signal_slices.shape)
            if par["state_save"] == False:
                signal_slices, time_stamps_slices = filter_low_entropy_slices(signal_slices, time_stamps_slices, print_parameters)
            
            print("3",signal_slices.shape)
            print_log(par, "Finished pre-processing! appending...")

            # Continue if no data.
            if signal_slices.shape[0] == 0: continue
        
            # Define metrics.
            last_stamp = 0 # Used in append_images for calculating time delta.
            next_percent = 10 # Variable helps reduce log output in print_progress().

            # Transform pre-processed signal slices. 
            for i, [sig, time_stamps] in enumerate(zip(signal_slices, time_stamps_slices)):
    
                next_percent = print_progress(i, signal_slices, sig_stack, next_percent, par)

                # Stop if enough data.
                if sig_stack.shape[0] >= par["max_images"]:
                    print_log(par,"max size of "f"{par['max_images']} reached, skipping!")
                    break

                if par["state_save"]:
                    sig, state = get_state(sig, par)
                else:
                    state = 1

                # Append transformed images to stack.
                state_stack, state_stack_tmp, sig_stack, sig_stack_tmp, timestamp_stack, timestamp_stack_tmp, last_stamp = append_images(state, state_stack, state_stack_tmp,
                                                                                               sig, sig_stack, sig_stack_tmp,
                                                                                               timestamp_stack, timestamp_stack_tmp,
                                                                                               time_stamps, last_stamp, par) 
        
        if sig_stack.shape[0] > 0:
            # Save images.
            group_path = f"{par['dataset_name']}/"f"{appliance}/"f"{building}"

            print("signal",sig_stack.shape)
            print("ts", timestamp_stack.shape)
            print("lab",state_stack.shape)
            
            # Update metrics.
            sig_stacked_per_appliance += sig_stack.shape[0]
            sig_stacked += sig_stack.shape[0]
            healthy_appliances.add(appliance)
            
            if par["sig_save"]:
                # Save source time series.
                store_many_hdf5(file_name, sig_stack, group_path, "sig", force_del="yes")

            if par["ts_save"]:
                # Save source time series.
                store_many_hdf5(file_name, timestamp_stack, group_path, "ts", force_del="yes")

            if par["state_save"]:
                # Save state of slice (on or off).
                store_many_hdf5(file_name, state_stack, group_path, "state", force_del="yes")
        
        else:
            print_log(par,"empty for building", building, "appliance", appliance)
  
        print_log(par, "finished building N", building)
    
    print_end_of_loop(sig_stacked_per_appliance, appliance, par)
    
print_end(sig_stacked, healthy_appliances, par)

dataset name:  iawe , date :  2022-03-21 10:05:40  
 using appliances:  ['air conditioner', 'unknown', 'wet appliance', 'motor', 'television', 'fridge', 'computer', 'washing machine', 'clothes iron'] 
 windows size in mins: 13 
 image size in pixels: 100 
 sample period: 6 
 number if images that are stacked together and in series(series!): 1 
 allowed max delta between images  600 
 save source timeseries: True 
 save signal: True 
 save state: True 
 manually select appliances: False 
 number of appliances: 9 
 number of buildings: 1 
 max number of images per appliance per building: 10000000 
 include multiple buildings (Y for Yes N for No): True 
 building selected: A 

creating new file! ...

 Starting air conditioner (1/9):

 Starting building 1
1 (11354, 130)
2 (776, 130)
3 (776, 130)
Finished pre-processing! appending...
processed: 10.05 % finished: 0.0 %
processed: 20.1 % finished: 0.0 %
processed: 30.03 % finished: 0.0 %
processed: 40.08 % finished: 0.0 %
processed: 50.13 % f